In [7]:
import numpy
from keras.datasets import cifar10
from matplotlib import pyplot
from scipy.misc import toimage

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

seed = 7
numpy.random.seed(seed)


(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train = X_train/255.0
X_test = X_test/255.0

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

num_classes = y_train.shape[1]
def baseline_model():
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, 
                            input_shape=(3, 32, 32),
                            border_mode = 'same', 
                            activation = 'relu',
                            W_constraint = maxnorm(3)))
    model.add(Dropout(0.2))
    model.add(Convolution2D(32, 3, 3,
                            activation = 'relu',
                            border_mode = 'same',
                            W_constraint = maxnorm(3)))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Flatten())
    model.add(Dense(512, activation = 'relu', W_constraint = maxnorm(3)))
    
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation = 'softmax'))
    
    epochs = 25
    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr = lrate, decay = decay, momentum = 0.9, nesterov=False)
    
    model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])
    print(model.summary())
    return model

def larger_model():
    model = Sequential()
    model.add(Convolution2D(32, 3, 3, input_shape=(3, 32, 32), activation = 'relu', border_mode = 'same'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Convolution2D(64, 3, 3, activation = 'relu', border_mode = 'same'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Convolution2D(128, 3, 3, activation = 'relu', border_mode = 'same'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    
    model.add(Flatten())
    
    model.add(Dropout(0.2))
    
    model.add(Dense(1024, activation = 'relu', W_constraint = maxnorm(3)))
    model.add(Dropout(0.2))
    
    model.add(Dense(512, activation='relu', W_constraint=maxnorm(3)))
    model.add(Dropout(0.2))
    
    model.add(Dense(num_classes, activation = 'softmax'))
    
    # Compile
    epochs = 25.0
    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr = lrate, decay = decay, momentum = 0.9, nesterov = False)
    model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])
    
    print model.summary()
    return model

In [ ]:
model = baseline_model()
model.fit(X_train, y_train,
          validation_data = (X_test, y_test),
          nb_epoch = 25,
          batch_size = 32)

scores = model.evaluate(X_test, y_test, verbose = 0)

print("Baseline score: %.2f" %(scores[1]*100))

In [10]:
model = larger_model()

model.fit(X_train, y_train, 
          validation_data = (X_test, y_test),
          nb_epoch = 25,
         batch_size = 100)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_13 (Convolution2D) (None, 32, 32, 32)    896         convolution2d_input_6[0][0]      
____________________________________________________________________________________________________
maxpooling2d_13 (MaxPooling2D)   (None, 32, 16, 16)    0           convolution2d_13[0][0]           
____________________________________________________________________________________________________
dropout_14 (Dropout)             (None, 32, 16, 16)    0           maxpooling2d_13[0][0]            
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 16, 16)    18496       dropout_14[0][0]                 
___________________________________________________________________________________________

In [9]:
scores = model.evaluate(X_test, y_test, verbose=0)

print ("Larger Score %.2f" %(scores[1]*100))

Larger Score 78.66
